In [ ]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  1696k      0 --:--:-- --:--:-- --:--:-- 1699k


# New section

In [ ]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [ ]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [ ]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [ ]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

In [ ]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [ ]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0142, 0.0191, 0.0233,  ..., 0.0163, 0.0365, 0.0480],
        [0.0330, 0.0092, 0.0085,  ..., 0.0494, 0.0009, 0.0344],
        [0.0267, 0.0220, 0.0034,  ..., 0.0286, 0.0011, 0.0168],
        ...,
        [0.0128, 0.0395, 0.0327,  ..., 0.0160, 0.0079, 0.0112],
        [0.0128, 0.0428, 0.0486,  ..., 0.0479, 0.0196, 0.0064],
        [0.0003, 0.0045, 0.0100,  ..., 0.0419, 0.0254, 0.0216]])
item_factors.weight tensor([[0.0354, 0.0306, 0.0142,  ..., 0.0288, 0.0438, 0.0492],
        [0.0399, 0.0358, 0.0153,  ..., 0.0314, 0.0380, 0.0234],
        [0.0076, 0.0324, 0.0050,  ..., 0.0247, 0.0223, 0.0312],
        ...,
        [0.0145, 0.0168, 0.0100,  ..., 0.0090, 0.0345, 0.0414],
        [0.0291, 0.0223, 0.0364,  ..., 0.0151, 0.0101, 0.0398],
        [0.0104, 0.0164, 0.0231,  ..., 0.0051, 0.0359, 0.0081]])


In [ ]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-12-dad152416852>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.065820924521702
iter #1 Loss: 4.743957787600871
iter #2 Loss: 2.473855237821637
iter #3 Loss: 1.7206896256069242
iter #4 Loss: 1.3456787594835165
iter #5 Loss: 1.1286448994415061
iter #6 Loss: 0.9916838670745114
iter #7 Loss: 0.9002449968593375
iter #8 Loss: 0.8372577323677576
iter #9 Loss: 0.7923995195533419
iter #10 Loss: 0.7595763130085117
iter #11 Loss: 0.7347544271328728
iter #12 Loss: 0.7163585440685907
iter #13 Loss: 0.7015351181102888
iter #14 Loss: 0.690684011593688
iter #15 Loss: 0.6815360497853478
iter #16 Loss: 0.6753166539944367
iter #17 Loss: 0.6698773710180055
iter #18 Loss: 0.6657152186114776
iter #19 Loss: 0.6627314503347208
iter #20 Loss: 0.6605093209876627
iter #21 Loss: 0.6588791715841608
iter #22 Loss: 0.6576023152107515
iter #23 Loss: 0.6564574240503578
iter #24 Loss: 0.6558264413763424
iter #25 Loss: 0.6549984224254105
iter #26 Loss: 0.653934413292989
iter #27 Loss: 0.6533403451445744
iter #28 Loss: 0.6518766656139781
iter #29 Loss: 0.65067781076

In [ ]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.2815,  0.6871,  1.5404,  ...,  1.1143,  1.4846,  1.4100],
        [ 1.4332, -0.3627,  0.8919,  ...,  1.2597,  1.6073,  1.2253],
        [ 1.2726,  0.6530,  1.0881,  ..., -1.1147,  2.1534, -2.4044],
        ...,
        [ 2.2274,  2.0693,  1.3771,  ...,  0.2053, -0.5591,  0.4093],
        [ 1.3621,  0.9902,  0.7449,  ...,  1.1791,  0.7659,  1.2561],
        [ 0.3995,  0.8395,  1.4265,  ...,  0.9968,  1.1305,  0.5173]],
       device='cuda:0')
item_factors.weight tensor([[ 0.2323,  0.4148,  0.4019,  ...,  0.3331,  0.5571,  0.6492],
        [ 0.6272, -0.2002,  0.4481,  ...,  0.7406,  0.5328,  0.4257],
        [ 0.4224,  0.5695,  0.3131,  ...,  0.4993,  0.2424,  0.1737],
        ...,
        [ 0.3227,  0.3261,  0.3211,  ...,  0.3191,  0.3442,  0.3523],
        [ 0.4194,  0.4122,  0.4250,  ...,  0.4017,  0.3976,  0.4243],
        [ 0.3856,  0.3909,  0.3971,  ...,  0.3790,  0.4101,  0.3838]],
       device='cuda:0')


In [ ]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [ ]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [ ]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Jurassic Park (1993)
	 Apollo 13 (1995)
	 Batman (1989)
	 Aladdin (1992)
	 Sixth Sense, The (1999)
	 Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
	 Men in Black (a.k.a. MIB) (1997)
	 Dances with Wolves (1990)
	 Beauty and the Beast (1991)
	 Die Hard (1988)
Cluster #1
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Star Wars: Episode IV - A New Hope (1977)
	 Braveheart (1995)
	 Fight Club (1999)
	 Toy Story (1995)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
	 Usual Suspects, The (1995)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Cluster #2
	 True Lies (1994)
	 Speed (1994)
	 Mask, The (1994)
	 Babe (1995)
	 Twister (1996)
	 Rock, The (1996)
	 Home Alone (1990)
	 Outbreak (1995)
	 Matrix Reloaded, The (2003)
	 Mummy, The (1999)
Cluster #3
	 Ghost (1990)
	 Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
	 Harry Potter and the Chamber of Se